In [ ]:
# --- Load the pairwise notebook (so this notebook is standalone)
from __future__ import annotations

from pathlib import Path
import nbformat
from IPython import get_ipython

def _exec_notebook(path: Path) -> None:
    """Execute all *code* cells of a notebook into the current IPython kernel."""
    nb = nbformat.read(path, as_version=4)
    ip = get_ipython()
    for cell in nb.cells:
        if cell.cell_type == 'code' and cell.source.strip():
            ip.run_cell(cell.source)


def _find_local_notebook(name: str) -> Path:
    """Find a notebook file by searching common locations."""
    candidates = []
    # 1) current working directory and a few parents
    p = Path.cwd().resolve()
    for _ in range(3):
        candidates.append(p / name)
        p = p.parent
    # 2) fallback: /mnt/data (common in sandboxes)
    candidates.append(Path('/mnt/data') / name)
    for c in candidates:
        if c.exists():
            return c
    raise FileNotFoundError(f"Cannot find {name}. Looked in: " + ", ".join(str(c) for c in candidates))

pairwise_nb = _find_local_notebook('did_multiplegt_stat_pairwise.ipynb')
if 'did_multiplegt_stat_pairwise' not in globals():
    if not pairwise_nb.exists():
        raise FileNotFoundError(
            "Cannot find 'did_multiplegt_stat_pairwise.ipynb' in the current folder. "
            "Put it next to this notebook, then Run All again."
        )
    _exec_notebook(pairwise_nb)
    print('Loaded pairwise from', pairwise_nb)

In [9]:
# --- Imports
import math
import numpy as np
import pandas as pd
from typing import Any, Dict, List, Optional


In [ ]:
def _balance_panel_fill(df: pd.DataFrame, id_col: str, t_col: str) -> pd.DataFrame:
    # Replicates make.pbalanced(..., balance.type="fill") (plm/R)
    # - adds missing rows ID x T con NA
    # - creates tsfilled_XX = 1 si fue “rellenado”
    # - re-maps T a 1..T en orden creciente de los valores originales
    df = df.copy()
    df["tsfilled_XX"] = 0

    times = pd.Series(df[t_col].dropna().unique()).sort_values().to_list()
    ids = pd.Series(df[id_col].dropna().unique()).to_list()

    mi = pd.MultiIndex.from_product([ids, times], names=[id_col, t_col])

    df_bal = (
        df.set_index([id_col, t_col])
          .reindex(mi)
          .reset_index()
    )
    df_bal["tsfilled_XX"] = df_bal["tsfilled_XX"].isna().astype(int)

    time_map = {t: i + 1 for i, t in enumerate(times)}
    df_bal[t_col] = df_bal[t_col].map(time_map).astype(int)

    return df_bal


def _se_from_phi(phi: pd.Series) -> float:
    # SE = sd(phi)/sqrt(n), ignorando NA.
    phi = pd.to_numeric(phi, errors="coerce").dropna()
    n = len(phi)
    if n <= 1:
        return np.nan
    return float(phi.std(ddof=1) / math.sqrt(n))


def _se_cluster_from_phi(ids_df: pd.DataFrame, cluster_col: str, phi_col: str, N_bar_c: float) -> float:
    # Cluster-robusta al estilo del main en R:
    #   SE = sd( sum_c Phi_c ) / sqrt(#clusters) / sqrt(N_bar_c)
    # donde Phi_c = suma de Phi por cluster, y N_bar_c = tamaño promedio del cluster (#IDs).
    tmp = ids_df[[cluster_col, phi_col]].copy()
    tmp[phi_col] = pd.to_numeric(tmp[phi_col], errors="coerce")

    s = tmp.groupby(cluster_col)[phi_col].sum(min_count=1).dropna()
    if len(s) <= 1:
        return np.nan
    if (not np.isfinite(N_bar_c)) or (N_bar_c <= 0):
        return np.nan

    return float(s.std(ddof=1) / math.sqrt(len(s)) / math.sqrt(float(N_bar_c)))





# Define the main aggregation function that mirrors the R `did_multiplegt_stat_main` logic.
def did_multiplegt_stat_main(
    # Parameter `df`: The input long-format panel DataFrame.
    df: pd.DataFrame,
    # Parameter `Y`: Name of the outcome column in `df`.
    Y: str,
    # Parameter `ID`: Name of the unit identifier column in `df`.
    ID: str,
    # Parameter `Time`: Name of the time-period column in `df`.
    Time: str,
    # Parameter `D`: Name of the treatment column in `df`.
    D: str,
    # Parameter `Z`: Optional instrument column name (only used for ivwaoss).
    Z: Optional[str],
    # Parameter `estimator`: List of estimators to compute: any of {'aoss','waoss','ivwaoss'}.
    estimator: List[str],
    # Parameter `estimation_method`: Estimation method passed to pairwise (e.g., 'dr', 'ra', 'ps').
    estimation_method: str,
    # Parameter `order`: Order for the local polynomial / regression components (passed to pairwise).
    order: int,
    # Parameter `noextrapolation`: If True, enforce support restrictions (passed to pairwise).
    noextrapolation: bool,
    # Parameter `placebo`: If True, also compute placebo estimates (pre-trends style).
    placebo: bool,
    # Parameter `switchers`: Which switchers to use (e.g., 'up', 'down', or None for both).
    switchers: Optional[str],
    # Parameter `disaggregate`: Whether to keep disaggregated outputs (kept for API parity).
    disaggregate: bool,
    # Parameter `aoss_vs_waoss`: If True, compute an AOSS vs WAOSS difference test.
    aoss_vs_waoss: bool,
    # Parameter `exact_match`: If True, enforce exact matching support restrictions (passed to pairwise).
    exact_match: bool,
    # Parameter `weight`: Optional weight column name; if None, uses 1.
    weight: Optional[str],
    # Parameter `cluster`: Optional clustering column name (must be coarser than ID).
    cluster: Optional[str],
    # Parameter `by_fd_opt`: Optional settings for by-first-difference logic (passed to pairwise).
    by_fd_opt: Optional[Any],
    # Parameter `other_treatments`: Optional list of additional treatment columns to control for (passed to pairwise).
    other_treatments: Optional[List[str]],
    # Parameter `legacy_r_phi_scale`: Controls the legacy rescaling of influence functions to match the R SE output.
    legacy_r_phi_scale: str = "off",  # "auto" | "on" | "off"
# Close the signature and declare the return type.
) -> Dict[str, Any]:
    # Main Python (high-fidelity) inspired by did_multiplegt_stat_main (R).
    # Compatibility:
    # - legacy_r_phi_scale="on" intenta detectar si pairwise devuelve SE más chico que sd/sqrt(n) y reescala Phi.
    # Flag whether AOSS is requested (1) or not (0).
    aoss_XX = int("aoss" in estimator)
    # Flag whether WAOSS is requested (1) or not (0).
    waoss_XX = int("waoss" in estimator)
    # Flag whether IVWAOSS is requested (1) or not (0).
    ivwaoss_XX = int("ivwaoss" in estimator)

    # select the required columns
    # Execute this statement as part of the main procedure.
    varlist: List[str] = []
    # Start a loop that iterates over the specified sequence.
    for v in [Y, ID, Time, D, Z, weight, cluster] + (other_treatments or []):
        # Conditional branch: execute the following block only if the condition is true.
        if v is not None and v not in varlist:
            # Execute this call (a helper/utility step needed for the main procedure).
            varlist.append(v)
    # Conditional branch: execute the following block only if the condition is true.
    if "partition_XX" in df.columns and "partition_XX" not in varlist:
        # Execute this call (a helper/utility step needed for the main procedure).
        varlist.append("partition_XX")
    # Compute and store `df`.
    df = df[varlist].copy()

    # map to *_XX
    # Build a mapping from internal standardized names (*_XX) to user-provided column names.
    mapping = {"Y_XX": Y, "ID_XX": ID, "T_XX": Time, "D_XX": D}
    # Conditional branch: execute the following block only if the condition is true.
    if Z is not None:
        # Execute this statement as part of the main procedure.
        mapping["Z_XX"] = Z
    # Conditional branch: execute the following block only if the condition is true.
    if weight is not None:
        # Execute this statement as part of the main procedure.
        mapping["weight_XX"] = weight
    # Conditional branch: execute the following block only if the condition is true.
    if cluster is not None:
        # Execute this statement as part of the main procedure.
        mapping["cluster_XX"] = cluster
    # Start a loop that iterates over the specified sequence.
    for new_col, old_col in mapping.items():
        # Conditional branch: execute the following block only if the condition is true.
        if old_col is not None:
            # Execute this statement as part of the main procedure.
            df[new_col] = df[old_col]

    # check cluster nestedness (si cluster==ID, ignorar)
    # Compute and store `n_clus_XX`.
    n_clus_XX = None
    # Conditional branch: execute the following block only if the condition is true.
    if cluster is not None:
        # Conditional branch: execute the following block only if the condition is true.
        if cluster == ID:
            # Compute and store `cluster`.
            cluster = None
            # Execute this call (a helper/utility step needed for the main procedure).
            df.drop(columns=["cluster_XX"], inplace=True, errors="ignore")
            # Execute this call (a helper/utility step needed for the main procedure).
            print("️ cluster == ID. Ignoro cluster (como en R).")
        # Fallback branch executed when none of the previous conditions matched.
        else:
            # Compute and store `g`.
            g = df.groupby("ID_XX")["cluster_XX"].nunique(dropna=True)
            # Conditional branch: execute the following block only if the condition is true.
            if (g > 1).any():
                # Execute this call (a helper/utility step needed for the main procedure).
                raise ValueError("La variable ID debe estar anidada dentro de cluster (cluster constante por ID).")
            # Compute and store `n_clus_XX`.
            n_clus_XX = int(df["cluster_XX"].nunique(dropna=True))

    # drop NAs en ID/T/D (y Z si IV)
    # Execute this call (a helper/utility step needed for the main procedure).
    df["to_drop_XX"] = df["T_XX"].isna() | df["D_XX"].isna() | df["ID_XX"].isna()
    # Compute and store `IV_req_XX`.
    IV_req_XX = 0
    # Conditional branch: execute the following block only if the condition is true.
    if ivwaoss_XX == 1:
        # Compute and store `IV_req_XX`.
        IV_req_XX = 1
        # Execute this call (a helper/utility step needed for the main procedure).
        df["to_drop_XX"] = df["to_drop_XX"] | df["Z_XX"].isna()
    # Compute and store `df`.
    df = df.loc[~df["to_drop_XX"]].copy()

    # balancear panel y remapear T
    # Compute and store `df`.
    df = _balance_panel_fill(df, id_col="ID_XX", t_col="T_XX")

    # pesos
    # Conditional branch: execute the following block only if the condition is true.
    if weight is None:
        # Execute this statement as part of the main procedure.
        df["weight_XX"] = 1.0
        # Execute this statement as part of the main procedure.
        df["weight_c_XX"] = 1.0
    # Fallback branch executed when none of the previous conditions matched.
    else:
        # Execute this call (a helper/utility step needed for the main procedure).
        df["weight_XX"] = df["weight_XX"].fillna(0.0).astype(float)
        # Execute this statement as part of the main procedure.
        df["weight_c_XX"] = 1.0
    # Conditional branch: execute the following block only if the condition is true.
    if cluster is not None:
        # Execute this call (a helper/utility step needed for the main procedure).
        df["weight_c_XX"] = df.groupby(["cluster_XX", "T_XX"])["weight_XX"].transform("sum").astype(float)

    # IDs dataframe
    # Create the per-unit table that will store influence-function components.
    IDs_XX = pd.DataFrame({"ID_XX": pd.Series(df["ID_XX"].unique()).sort_values().to_numpy()})
    # Conditional branch: execute the following block only if the condition is true.
    if cluster is not None:
        # Create the per-unit table that will store influence-function components.
        IDs_XX = IDs_XX.merge(df.groupby("ID_XX")["cluster_XX"].mean().reset_index(), on="ID_XX", how="left")

    # Compute the maximum (re-mapped) time index in the balanced panel.
    max_T_XX = int(df["T_XX"].max())

    # scalars
    # Execute this statement as part of the main procedure.
    scalars: Dict[str, Any] = dict(
        # Compute and store `PS_sum_XX`.
        PS_sum_XX=0.0, delta_1_1_XX=0.0,
        # Compute and store `E_abs_delta_D_sum_XX`.
        E_abs_delta_D_sum_XX=0.0, delta_2_1_XX=0.0,
        # Compute and store `denom_delta_IV_sum_XX`.
        denom_delta_IV_sum_XX=0.0, delta_3_1_XX=0.0,
        # Compute and store `N_Switchers_1_1_XX`.
        N_Switchers_1_1_XX=0.0, N_Stayers_1_1_XX=0.0,
        # Compute and store `N_Switchers_2_1_XX`.
        N_Switchers_2_1_XX=0.0, N_Stayers_2_1_XX=0.0,
        # Compute and store `N_Switchers_3_1_XX`.
        N_Switchers_3_1_XX=0.0, N_Stayers_3_1_XX=0.0,
        # Compute and store `N_drop_total_XX`.
        N_drop_total_XX=0.0, N_drop_total_C_XX=0.0,
        # Compute and store `IV_req_XX`.
        IV_req_XX=float(IV_req_XX),
    # Execute this statement as part of the main procedure.
    )
    # Conditional branch: execute the following block only if the condition is true.
    if bool(placebo):
        # Execute this statement as part of the main procedure.
        scalars.update(dict(
            # Compute and store `PS_sum_pl_XX`.
            PS_sum_pl_XX=0.0, delta_1_1_pl_XX=0.0,
            # Compute and store `E_abs_delta_D_sum_pl_XX`.
            E_abs_delta_D_sum_pl_XX=0.0, delta_2_1_pl_XX=0.0,
            # Compute and store `denom_delta_IV_sum_pl_XX`.
            denom_delta_IV_sum_pl_XX=0.0, delta_3_1_pl_XX=0.0,
            # Compute and store `N_Switchers_1_1_pl_XX`.
            N_Switchers_1_1_pl_XX=0.0, N_Stayers_1_1_pl_XX=0.0,
            # Compute and store `N_Switchers_2_1_pl_XX`.
            N_Switchers_2_1_pl_XX=0.0, N_Stayers_2_1_pl_XX=0.0,
            # Compute and store `N_Switchers_3_1_pl_XX`.
            N_Switchers_3_1_pl_XX=0.0, N_Stayers_3_1_pl_XX=0.0,
        # Execute this statement as part of the main procedure.
        ))

    # auto legacy scaling: ajustar Phi_* para igualar el SE que pairwise reporta
    # Execute this statement as part of the main procedure.
    def _maybe_rescale_phi_auto(to_add: pd.DataFrame, scalars_out: Dict[str, Any], p: int, pl_suffix: str) -> pd.DataFrame:
        # Conditional branch: execute the following block only if the condition is true.
        if legacy_r_phi_scale == "off":
            # Return the final output dictionary to the caller.
            return to_add

        # Start a loop that iterates over the specified sequence.
        for j, enabled in [(2, waoss_XX), (1, aoss_XX), (3, ivwaoss_XX)]:
            # Conditional branch: execute the following block only if the condition is true.
            if enabled != 1:
                # Execute this statement as part of the main procedure.
                continue
            # Compute and store `phi_col`.
            phi_col = f"Phi_{j}_{p}{pl_suffix}_XX"
            # Compute and store `se_key`.
            se_key  = f"sd_delta_{j}_{p}{pl_suffix}_XX"
            # Conditional branch: execute the following block only if the condition is true.
            if phi_col in to_add.columns and se_key in scalars_out:
                # Compute and store `se_target`.
                se_target = float(scalars_out.get(se_key, np.nan))
                # Conditional branch: execute the following block only if the condition is true.
                if not np.isfinite(se_target) or se_target <= 0:
                    # Execute this statement as part of the main procedure.
                    continue
                # Compute and store `se_direct`.
                se_direct = _se_from_phi(to_add[phi_col])
                # Conditional branch: execute the following block only if the condition is true.
                if not np.isfinite(se_direct) or se_direct <= 0:
                    # Execute this statement as part of the main procedure.
                    continue

                # Conditional branch: execute the following block only if the condition is true.
                if legacy_r_phi_scale == "on":
                    # Compute and store `W_key`.
                    W_key = f"W{pl_suffix}_XX" if f"W{pl_suffix}_XX" in scalars_out else "W_XX"
                    # Compute and store `W`.
                    W = float(scalars_out.get(W_key, np.nan))
                    # Compute and store `scale`.
                    scale = (1.0 / math.sqrt(W)) if (np.isfinite(W) and W > 0) else (se_target / se_direct)
                # Fallback branch executed when none of the previous conditions matched.
                else:
                    # Compute and store `ratio`.
                    ratio = se_target / se_direct
                    # Conditional branch: execute the following block only if the condition is true.
                    if abs(ratio - 1.0) < 0.10:
                        # Return the final output dictionary to the caller.
                        return to_add
                    # Compute and store `scale`.
                    scale = ratio

                # Start a loop that iterates over the specified sequence.
                for jj in (1, 2, 3):
                    # Compute and store `c`.
                    c = f"Phi_{jj}_{p}{pl_suffix}_XX"
                    # Conditional branch: execute the following block only if the condition is true.
                    if c in to_add.columns:
                        # Execute this statement as part of the main procedure.
                        to_add[c] = pd.to_numeric(to_add[c], errors="coerce") * scale
                # Return the final output dictionary to the caller.
                return to_add
        # Return the final output dictionary to the caller.
        return to_add

    # LOOP principal
    # Start a loop that iterates over the specified sequence.
    for p in range(2, max_T_XX + 1):
        # Compute and store `est_out`.
        est_out = did_multiplegt_stat_pairwise(
            # Compute and store `df`.
            df=df,
            # Compute and store `Y`.
            Y="Y_XX", ID="ID_XX", Time="T_XX", D="D_XX",
            # Compute and store `Z`.
            Z=("Z_XX" if Z is not None else None),
            # Compute and store `estimator`.
            estimator=estimator,
            # Compute and store `order`.
            order=order,
            # Compute and store `noextrapolation`.
            noextrapolation=noextrapolation,
            # Compute and store `weight`.
            weight="weight_XX",
            # Compute and store `switchers`.
            switchers=switchers,
            # Compute and store `pairwise`.
            pairwise=p,
            # Compute and store `IDs`.
            IDs=IDs_XX,
            # Compute and store `aoss`.
            aoss=aoss_XX, waoss=waoss_XX, ivwaoss=ivwaoss_XX,
            # Compute and store `estimation_method`.
            estimation_method=estimation_method,
            # Initialize the dictionary of scalar accumulators used across pairwise computations.
            scalars=scalars,
            # Compute and store `placebo`.
            placebo=0,  # FIXED: use 0 instead of False
            # Compute and store `exact_match`.
            exact_match=exact_match,
            # Compute and store `cluster`.
            cluster=cluster,
            # Compute and store `by_fd_opt`.
            by_fd_opt=by_fd_opt,
            # Compute and store `other_treatments`.
            other_treatments=other_treatments,
        # Execute this statement as part of the main procedure.
        )
        # Compute and store `to_add`.
        to_add = est_out.get("to_add", None)
        # Initialize the dictionary of scalar accumulators used across pairwise computations.
        scalars = est_out["scalars"]

        # Conditional branch: execute the following block only if the condition is true.
        if to_add is not None and isinstance(to_add, pd.DataFrame) and len(to_add) > 0:
            # Compute and store `to_add`.
            to_add = _maybe_rescale_phi_auto(to_add, scalars, p=p, pl_suffix="")
            # Create the per-unit table that will store influence-function components.
            IDs_XX = IDs_XX.merge(to_add, on="ID_XX", how="left").sort_values("ID_XX").reset_index(drop=True)

        # Conditional branch: execute the following block only if the condition is true.
        if aoss_XX == 1:
            # Execute this call (a helper/utility step needed for the main procedure).
            scalars["delta_1_1_XX"] += scalars.get(f"P_{p}_XX", 0.0) * scalars.get(f"delta_1_{p}_XX", 0.0)
            # Conditional branch: execute the following block only if the condition is true.
            if scalars.get(f"N_Stayers_1_{p}_XX", 0.0) > 1:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["N_Switchers_1_1_XX"] += scalars.get(f"N_Switchers_1_{p}_XX", 0.0)
            # Conditional branch: execute the following block only if the condition is true.
            if scalars.get(f"N_Switchers_1_{p}_XX", 0.0) > 0:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["N_Stayers_1_1_XX"] += scalars.get(f"N_Stayers_1_{p}_XX", 0.0)

        # Conditional branch: execute the following block only if the condition is true.
        if waoss_XX == 1:
            # Execute this call (a helper/utility step needed for the main procedure).
            scalars["delta_2_1_XX"] += scalars.get(f"E_abs_delta_D_{p}_XX", 0.0) * scalars.get(f"delta_2_{p}_XX", 0.0)
            # Conditional branch: execute the following block only if the condition is true.
            if scalars.get(f"N_Stayers_2_{p}_XX", 0.0) > 1:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["N_Switchers_2_1_XX"] += scalars.get(f"N_Switchers_2_{p}_XX", 0.0)
            # Conditional branch: execute the following block only if the condition is true.
            if scalars.get(f"N_Switchers_2_{p}_XX", 0.0) > 0:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["N_Stayers_2_1_XX"] += scalars.get(f"N_Stayers_2_{p}_XX", 0.0)

        # Conditional branch: execute the following block only if the condition is true.
        if ivwaoss_XX == 1:
            # Execute this call (a helper/utility step needed for the main procedure).
            scalars["delta_3_1_XX"] += scalars.get(f"denom_delta_IV_{p}_XX", 0.0) * scalars.get(f"delta_3_{p}_XX", 0.0)
            # Conditional branch: execute the following block only if the condition is true.
            if scalars.get(f"N_Stayers_3_{p}_XX", 0.0) > 1:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["N_Switchers_3_1_XX"] += scalars.get(f"N_Switchers_3_{p}_XX", 0.0)
            # Conditional branch: execute the following block only if the condition is true.
            if scalars.get(f"N_Switchers_3_{p}_XX", 0.0) > 0:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["N_Stayers_3_1_XX"] += scalars.get(f"N_Stayers_3_{p}_XX", 0.0)

    # LOOP placebo
    # Conditional branch: execute the following block only if the condition is true.
    if bool(placebo):
        # FIXED: placebo_index = 1 (for now only support single placebo)
        # In Stata: forvalues placebo_index = 1/`placebo'
        placebo_index = 1
        
        # FIXED: Loop starts at p = 2 + placebo_index (for placebo_index=1, starts at 3)
        # In Stata: forvalues p = `=2+`placebo_index''/`=max_T'
        for p in range(2 + placebo_index, max_T_XX + 1):
            # Compute and store `est_out`.
            est_out = did_multiplegt_stat_pairwise(
                # Compute and store `df`.
                df=df,
                # Compute and store `Y`.
                Y="Y_XX", ID="ID_XX", Time="T_XX", D="D_XX",
                # Compute and store `Z`.
                Z=("Z_XX" if Z is not None else None),
                # Compute and store `estimator`.
                estimator=estimator,
                # Compute and store `order`.
                order=order,
                # Compute and store `noextrapolation`.
                noextrapolation=noextrapolation,
                # Compute and store `weight`.
                weight="weight_XX",
                # Compute and store `switchers`.
                switchers=switchers,
                # Compute and store `pairwise`.
                pairwise=p,
                # Compute and store `IDs`.
                IDs=IDs_XX,
                # Compute and store `aoss`.
                aoss=aoss_XX, waoss=waoss_XX, ivwaoss=ivwaoss_XX,
                # Compute and store `estimation_method`.
                estimation_method=estimation_method,
                # Initialize the dictionary of scalar accumulators used across pairwise computations.
                scalars=scalars,
                # FIXED: pass placebo index (int) instead of True
                placebo=placebo_index,
                # Compute and store `exact_match`.
                exact_match=exact_match,
                # Compute and store `cluster`.
                cluster=cluster,
                # Compute and store `by_fd_opt`.
                by_fd_opt=by_fd_opt,
                # Compute and store `other_treatments`.
                other_treatments=other_treatments,
            # Execute this statement as part of the main procedure.
            )
            # Compute and store `to_add`.
            to_add = est_out.get("to_add", None)
            # Initialize the dictionary of scalar accumulators used across pairwise computations.
            scalars = est_out["scalars"]

            # Conditional branch: execute the following block only if the condition is true.
            if to_add is not None and isinstance(to_add, pd.DataFrame) and len(to_add) > 0:
                # Compute and store `to_add`.
                to_add = _maybe_rescale_phi_auto(to_add, scalars, p=p, pl_suffix="_pl")
                # Create the per-unit table that will store influence-function components.
                IDs_XX = IDs_XX.merge(to_add, on="ID_XX", how="left").sort_values("ID_XX").reset_index(drop=True)

            # Conditional branch: execute the following block only if the condition is true.
            if aoss_XX == 1:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["delta_1_1_pl_XX"] += scalars.get(f"P_{p}_pl_XX", 0.0) * scalars.get(f"delta_1_{p}_pl_XX", 0.0)
                if scalars.get(f"N_Stayers_1_{p}_pl_XX", 0.0) > 1:
                    scalars["N_Switchers_1_1_pl_XX"] += scalars.get(f"N_Switchers_1_{p}_pl_XX", 0.0)
                if scalars.get(f"N_Switchers_1_{p}_pl_XX", 0.0) > 0:
                    scalars["N_Stayers_1_1_pl_XX"] += scalars.get(f"N_Stayers_1_{p}_pl_XX", 0.0)
            # Conditional branch: execute the following block only if the condition is true.
            if waoss_XX == 1:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["delta_2_1_pl_XX"] += scalars.get(f"E_abs_delta_D_{p}_pl_XX", 0.0) * scalars.get(f"delta_2_{p}_pl_XX", 0.0)
                if scalars.get(f"N_Stayers_2_{p}_pl_XX", 0.0) > 1:
                    scalars["N_Switchers_2_1_pl_XX"] += scalars.get(f"N_Switchers_2_{p}_pl_XX", 0.0)
                if scalars.get(f"N_Switchers_2_{p}_pl_XX", 0.0) > 0:
                    scalars["N_Stayers_2_1_pl_XX"] += scalars.get(f"N_Stayers_2_{p}_pl_XX", 0.0)
            # Conditional branch: execute the following block only if the condition is true.
            if ivwaoss_XX == 1:
                # Execute this call (a helper/utility step needed for the main procedure).
                scalars["delta_3_1_pl_XX"] += scalars.get(f"denom_delta_IV_{p}_pl_XX", 0.0) * scalars.get(f"delta_3_{p}_pl_XX", 0.0)
                if scalars.get(f"N_Stayers_3_{p}_pl_XX", 0.0) > 1:
                    scalars["N_Switchers_3_1_pl_XX"] += scalars.get(f"N_Switchers_3_{p}_pl_XX", 0.0)
                if scalars.get(f"N_Switchers_3_{p}_pl_XX", 0.0) > 0:
                    scalars["N_Stayers_3_1_pl_XX"] += scalars.get(f"N_Stayers_3_{p}_pl_XX", 0.0)

    # dividir por sumas
    # Conditional branch: execute the following block only if the condition is true.
    if aoss_XX == 1 and scalars.get("PS_sum_XX", 0.0) != 0:
        # Execute this statement as part of the main procedure.
        scalars["delta_1_1_XX"] /= scalars["PS_sum_XX"]
        # Conditional branch: execute the following block only if the condition is true.
        if bool(placebo) and scalars.get("PS_sum_pl_XX", 0.0) != 0:
            # Execute this statement as part of the main procedure.
            scalars["delta_1_1_pl_XX"] /= scalars["PS_sum_pl_XX"]

    # Conditional branch: execute the following block only if the condition is true.
    if waoss_XX == 1 and scalars.get("E_abs_delta_D_sum_XX", 0.0) != 0:
        # Execute this statement as part of the main procedure.
        scalars["delta_2_1_XX"] /= scalars["E_abs_delta_D_sum_XX"]
        # Conditional branch: execute the following block only if the condition is true.
        if bool(placebo) and scalars.get("E_abs_delta_D_sum_pl_XX", 0.0) != 0:
            # Execute this statement as part of the main procedure.
            scalars["delta_2_1_pl_XX"] /= scalars["E_abs_delta_D_sum_pl_XX"]

    # Conditional branch: execute the following block only if the condition is true.
    if ivwaoss_XX == 1 and scalars.get("denom_delta_IV_sum_XX", 0.0) != 0:
        # Execute this statement as part of the main procedure.
        scalars["delta_3_1_XX"] /= scalars["denom_delta_IV_sum_XX"]
        # Conditional branch: execute the following block only if the condition is true.
        if bool(placebo) and scalars.get("denom_delta_IV_sum_pl_XX", 0.0) != 0:
            # Execute this statement as part of the main procedure.
            scalars["delta_3_1_pl_XX"] /= scalars["denom_delta_IV_sum_pl_XX"]

    # IF agregadas
    # Start a loop that iterates over the specified sequence.
    for i in (1, 2, 3):
        # Execute this statement as part of the main procedure.
        IDs_XX[f"Phi_{i}_XX"] = 0.0
        # Conditional branch: execute the following block only if the condition is true.
        if bool(placebo):
            # Execute this statement as part of the main procedure.
            IDs_XX[f"Phi_{i}_pl_XX"] = 0.0

    # Start a loop that iterates over the specified sequence.
    for p in range(2, max_T_XX + 1):
        # Compute and store `non_missing`.
        non_missing = int(scalars.get(f"non_missing_{p}_XX", 0) == 1)

        # Conditional branch: execute the following block only if the condition is true.
        if aoss_XX == 1 and non_missing == 1 and (f"Phi_1_{p}_XX" in IDs_XX.columns) and (f"S_{p}_XX" in IDs_XX.columns):
            # Compute and store `Phi_p`.
            Phi_p = pd.to_numeric(IDs_XX[f"Phi_1_{p}_XX"], errors="coerce")
            # Compute and store `S_p`.
            S_p = pd.to_numeric(IDs_XX[f"S_{p}_XX"], errors="coerce")
            # Compute and store `P_p`.
            P_p = float(scalars.get(f"P_{p}_XX", 0.0))
            # Compute and store `delta_p`.
            delta_p = float(scalars.get(f"delta_1_{p}_XX", np.nan))
            # Compute and store `denom`.
            denom = float(scalars.get("PS_sum_XX", np.nan))
            # Conditional branch: execute the following block only if the condition is true.
            if np.isfinite(denom) and denom != 0:
                # Compute and store `adj`.
                adj = (P_p * Phi_p + (delta_p - float(scalars.get("delta_1_1_XX", np.nan))) * (S_p - P_p)) / denom
                # Execute this call (a helper/utility step needed for the main procedure).
                IDs_XX["Phi_1_XX"] = IDs_XX["Phi_1_XX"] + adj.fillna(0.0)

        # Conditional branch: execute the following block only if the condition is true.
        if waoss_XX == 1 and non_missing == 1 and (f"Phi_2_{p}_XX" in IDs_XX.columns) and (f"abs_delta_D_{p}_XX" in IDs_XX.columns):
            # Compute and store `Phi_p`.
            Phi_p = pd.to_numeric(IDs_XX[f"Phi_2_{p}_XX"], errors="coerce")
            # Compute and store `abs_p`.
            abs_p = pd.to_numeric(IDs_XX[f"abs_delta_D_{p}_XX"], errors="coerce")
            # Compute and store `E_abs_p`.
            E_abs_p = float(scalars.get(f"E_abs_delta_D_{p}_XX", 0.0))
            # Compute and store `delta_p`.
            delta_p = float(scalars.get(f"delta_2_{p}_XX", np.nan))
            # Compute and store `denom`.
            denom = float(scalars.get("E_abs_delta_D_sum_XX", np.nan))
            # Conditional branch: execute the following block only if the condition is true.
            if np.isfinite(denom) and denom != 0:
                # Compute and store `adj`.
                adj = (E_abs_p * Phi_p + (delta_p - float(scalars.get("delta_2_1_XX", np.nan))) * (abs_p - E_abs_p)) / denom
                # Execute this call (a helper/utility step needed for the main procedure).
                IDs_XX["Phi_2_XX"] = IDs_XX["Phi_2_XX"] + adj.fillna(0.0)

        # Conditional branch: execute the following block only if the condition is true.
        if ivwaoss_XX == 1 and non_missing == 1 and (f"Phi_3_{p}_XX" in IDs_XX.columns) and (f"inner_sum_IV_denom_{p}_XX" in IDs_XX.columns):
            # Compute and store `Phi_p`.
            Phi_p = pd.to_numeric(IDs_XX[f"Phi_3_{p}_XX"], errors="coerce")
            # Compute and store `inn_p`.
            inn_p = pd.to_numeric(IDs_XX[f"inner_sum_IV_denom_{p}_XX"], errors="coerce")
            # Compute and store `denom_p`.
            denom_p = float(scalars.get(f"denom_delta_IV_{p}_XX", 0.0))
            # Compute and store `delta_p`.
            delta_p = float(scalars.get(f"delta_3_{p}_XX", np.nan))
            # Compute and store `denom`.
            denom = float(scalars.get("denom_delta_IV_sum_XX", np.nan))
            # Conditional branch: execute the following block only if the condition is true.
            if np.isfinite(denom) and denom != 0:
                # Compute and store `adj`.
                adj = (denom_p * Phi_p + (delta_p - float(scalars.get("delta_3_1_XX", np.nan))) * (inn_p - denom_p)) / denom
                # Execute this call (a helper/utility step needed for the main procedure).
                IDs_XX["Phi_3_XX"] = IDs_XX["Phi_3_XX"] + adj.fillna(0.0)

    # Conditional branch: execute the following block only if the condition is true.
    if bool(placebo):
        # Start a loop that iterates over the specified sequence.
        for p in range(3, max_T_XX + 1):
            # Compute and store `non_missing`.
            non_missing = int(scalars.get(f"non_missing_{p}_pl_XX", 0) == 1)

            # Conditional branch: execute the following block only if the condition is true.
            if aoss_XX == 1 and non_missing == 1 and (f"Phi_1_{p}_pl_XX" in IDs_XX.columns) and (f"S_{p}_pl_XX" in IDs_XX.columns):
                # Compute and store `Phi_p`.
                Phi_p = pd.to_numeric(IDs_XX[f"Phi_1_{p}_pl_XX"], errors="coerce")
                # Compute and store `S_p`.
                S_p = pd.to_numeric(IDs_XX[f"S_{p}_pl_XX"], errors="coerce")
                # Compute and store `P_p`.
                P_p = float(scalars.get(f"P_{p}_pl_XX", 0.0))
                # Compute and store `delta_p`.
                delta_p = float(scalars.get(f"delta_1_{p}_pl_XX", np.nan))
                # Compute and store `denom`.
                denom = float(scalars.get("PS_sum_pl_XX", np.nan))
                # Conditional branch: execute the following block only if the condition is true.
                if np.isfinite(denom) and denom != 0:
                    # Compute and store `adj`.
                    adj = (P_p * Phi_p + (delta_p - float(scalars.get("delta_1_1_pl_XX", np.nan))) * (S_p - P_p)) / denom
                    # Execute this call (a helper/utility step needed for the main procedure).
                    IDs_XX["Phi_1_pl_XX"] = IDs_XX["Phi_1_pl_XX"] + adj.fillna(0.0)

            # Conditional branch: execute the following block only if the condition is true.
            if waoss_XX == 1 and non_missing == 1 and (f"Phi_2_{p}_pl_XX" in IDs_XX.columns) and (f"abs_delta_D_{p}_pl_XX" in IDs_XX.columns):
                # Compute and store `Phi_p`.
                Phi_p = pd.to_numeric(IDs_XX[f"Phi_2_{p}_pl_XX"], errors="coerce")
                # Compute and store `abs_p`.
                abs_p = pd.to_numeric(IDs_XX[f"abs_delta_D_{p}_pl_XX"], errors="coerce")
                # Compute and store `E_abs_p`.
                E_abs_p = float(scalars.get(f"E_abs_delta_D_{p}_pl_XX", 0.0))
                # Compute and store `delta_p`.
                delta_p = float(scalars.get(f"delta_2_{p}_pl_XX", np.nan))
                # Compute and store `denom`.
                denom = float(scalars.get("E_abs_delta_D_sum_pl_XX", np.nan))
                # Conditional branch: execute the following block only if the condition is true.
                if np.isfinite(denom) and denom != 0:
                    # Compute and store `adj`.
                    adj = (E_abs_p * Phi_p + (delta_p - float(scalars.get("delta_2_1_pl_XX", np.nan))) * (abs_p - E_abs_p)) / denom
                    # Execute this call (a helper/utility step needed for the main procedure).
                    IDs_XX["Phi_2_pl_XX"] = IDs_XX["Phi_2_pl_XX"] + adj.fillna(0.0)

            # Conditional branch: execute the following block only if the condition is true.
            if ivwaoss_XX == 1 and non_missing == 1 and (f"Phi_3_{p}_pl_XX" in IDs_XX.columns) and (f"inner_sum_IV_denom_{p}_pl_XX" in IDs_XX.columns):
                # Compute and store `Phi_p`.
                Phi_p = pd.to_numeric(IDs_XX[f"Phi_3_{p}_pl_XX"], errors="coerce")
                # Compute and store `inn_p`.
                inn_p = pd.to_numeric(IDs_XX[f"inner_sum_IV_denom_{p}_pl_XX"], errors="coerce")
                # Compute and store `denom_p`.
                denom_p = float(scalars.get(f"denom_delta_IV_{p}_pl_XX", 0.0))
                # Compute and store `delta_p`.
                delta_p = float(scalars.get(f"delta_3_{p}_pl_XX", np.nan))
                # Compute and store `denom`.
                denom = float(scalars.get("denom_delta_IV_sum_pl_XX", np.nan))
                # Conditional branch: execute the following block only if the condition is true.
                if np.isfinite(denom) and denom != 0:
                    # Compute and store `adj`.
                    adj = (denom_p * Phi_p + (delta_p - float(scalars.get("delta_3_1_pl_XX", np.nan))) * (inn_p - denom_p)) / denom
                    # Execute this call (a helper/utility step needed for the main procedure).
                    IDs_XX["Phi_3_pl_XX"] = IDs_XX["Phi_3_pl_XX"] + adj.fillna(0.0)

    # SE/CI agregados
    # Conditional branch: execute the following block only if the condition is true.
    if cluster is not None:
        # Compute and store `tmp`.
        tmp = IDs_XX[["ID_XX", "cluster_XX"]].drop_duplicates()
        # Compute and store `N_bar_c_XX`.
        N_bar_c_XX = float(tmp.groupby("cluster_XX")["ID_XX"].size().mean())
    # Fallback branch executed when none of the previous conditions matched.
    else:
        # Compute and store `N_bar_c_XX`.
        N_bar_c_XX = np.nan

    # Start a loop that iterates over the specified sequence.
    for j in (1, 2, 3):
        # Compute and store `enabled`.
        enabled = {1: aoss_XX, 2: waoss_XX, 3: ivwaoss_XX}[j]
        # Conditional branch: execute the following block only if the condition is true.
        if enabled != 1:
            # Execute this statement as part of the main procedure.
            continue
        # Compute and store `phi_col`.
        phi_col = f"Phi_{j}_XX"
        # Compute and store `se`.
        se = _se_from_phi(IDs_XX[phi_col]) if cluster is None else _se_cluster_from_phi(IDs_XX, "cluster_XX", phi_col, N_bar_c_XX)
        # Execute this statement as part of the main procedure.
        scalars[f"sd_delta_{j}_1_XX"] = se
        # Execute this statement as part of the main procedure.
        scalars[f"LB_{j}_1_XX"] = float(scalars.get(f"delta_{j}_1_XX", np.nan)) - 1.96 * se
        # Execute this statement as part of the main procedure.
        scalars[f"UB_{j}_1_XX"] = float(scalars.get(f"delta_{j}_1_XX", np.nan)) + 1.96 * se

        # Conditional branch: execute the following block only if the condition is true.
        if bool(placebo):
            # Compute and store `phi_col`.
            phi_col = f"Phi_{j}_pl_XX"
            # Compute and store `se`.
            se = _se_from_phi(IDs_XX[phi_col]) if cluster is None else _se_cluster_from_phi(IDs_XX, "cluster_XX", phi_col, N_bar_c_XX)
            # Execute this statement as part of the main procedure.
            scalars[f"sd_delta_{j}_1_pl_XX"] = se
            # Execute this statement as part of the main procedure.
            scalars[f"LB_{j}_1_pl_XX"] = float(scalars.get(f"delta_{j}_1_pl_XX", np.nan)) - 1.96 * se
            # Execute this statement as part of the main procedure.
            scalars[f"UB_{j}_1_pl_XX"] = float(scalars.get(f"delta_{j}_1_pl_XX", np.nan)) + 1.96 * se

    # tabla output
    # Compute and store `estims`.
    estims = ["aoss", "waoss", "ivwaoss"]
    # Compute and store `colnames`.
    colnames = ["Estimate", "SE", "LB CI", "UB CI", "Switchers", "Stayers"]

    # Compute and store `ret`.
    ret = np.full((3 * max_T_XX, 6), np.nan, dtype=float)
    # Execute this statement as part of the main procedure.
    rown: List[str] = []
    # Start a loop that iterates over the specified sequence.
    for j, est in enumerate(estims, start=1):
        # Compute and store `enabled`.
        enabled = {"aoss": aoss_XX, "waoss": waoss_XX, "ivwaoss": ivwaoss_XX}[est]
        # Start a loop that iterates over the specified sequence.
        for p in range(1, max_T_XX + 1):
            # Execute this call (a helper/utility step needed for the main procedure).
            rown.append(est.upper() if p == 1 else f"{est}_{p}")
            # Conditional branch: execute the following block only if the condition is true.
            if enabled != 1:
                # Execute this statement as part of the main procedure.
                continue
            # Conditional branch: execute the following block only if the condition is true.
            if p == 1:
                # Compute and store `delta`.
                delta = scalars.get(f"delta_{j}_1_XX", np.nan)
                # Compute and store `se`.
                se = scalars.get(f"sd_delta_{j}_1_XX", np.nan)
                # Compute and store `lb`.
                lb = scalars.get(f"LB_{j}_1_XX", np.nan)
                # Compute and store `ub`.
                ub = scalars.get(f"UB_{j}_1_XX", np.nan)
                # Compute and store `ns`.
                ns = scalars.get(f"N_Switchers_{j}_1_XX", np.nan)
                # Compute and store `nt`.
                nt = scalars.get(f"N_Stayers_{j}_1_XX", np.nan)
            # Fallback branch executed when none of the previous conditions matched.
            else:
                # Compute and store `delta`.
                delta = scalars.get(f"delta_{j}_{p}_XX", np.nan)
                # Compute and store `ns`.
                ns = scalars.get(f"N_Switchers_{j}_{p}_XX", np.nan)
                # Compute and store `nt`.
                nt = scalars.get(f"N_Stayers_{j}_{p}_XX", np.nan)
                # Compute and store `phi_col`.
                phi_col = f"Phi_{j}_{p}_XX"
                # Conditional branch: execute the following block only if the condition is true.
                if cluster is None and phi_col in IDs_XX.columns:
                    # Compute and store `se`.
                    se = _se_from_phi(IDs_XX[phi_col])
                    # Compute and store `lb`.
                    lb = delta - 1.96 * se if np.isfinite(se) and np.isfinite(delta) else np.nan
                    # Compute and store `ub`.
                    ub = delta + 1.96 * se if np.isfinite(se) and np.isfinite(delta) else np.nan
                # Fallback branch executed when none of the previous conditions matched.
                else:
                    # Compute and store `se`.
                    se = scalars.get(f"sd_delta_{j}_{p}_XX", np.nan)
                    # Compute and store `lb`.
                    lb = scalars.get(f"LB_{j}_{p}_XX", np.nan)
                    # Compute and store `ub`.
                    ub = scalars.get(f"UB_{j}_{p}_XX", np.nan)
            # Execute this statement as part of the main procedure.
            ret[(j - 1) * max_T_XX + (p - 1), :] = [delta, se, lb, ub, ns, nt]

    # Compute and store `out_table`.
    out_table = pd.DataFrame(ret, index=rown, columns=colnames)
    # ----------------------------
    # Pack outputs (R-like object)
    # ----------------------------
    waoss_method_label_map = {
        "ra": "Regression Adjustment",
        "ps": "Propensity Score",
        "dr": "Doubly Robust",
    }
    common_support_label = "No Extrapolation" if bool(noextrapolation) else "Extrapolation"

    out: Dict[str, Any] = {
        "table": out_table,
        "pairs": int(max_T_XX),
        "N": int(df["ID_XX"].nunique()),
        "WAOSS Method": waoss_method_label_map.get(
            str(estimation_method).lower() if estimation_method is not None else None,
            str(estimation_method),
        ),
        "Polynomial Order": int(order),
        "Common Support": common_support_label,
    }

    if n_clus_XX is not None:
        out["n_clusters"] = int(n_clus_XX)

    # ----------------------------

    # --- Placebo counts for the aggregated (p=1) placebo table ---
    # In the pairwise code, counts are stored for each p>=3 as:
    #   N_Switchers_{j}_{p}_pl_XX and N_Stayers_{j}_{p}_pl_XX
    # But the placebo table (p=1) expects the aggregated totals:
    #   N_Switchers_{j}_1_pl_XX and N_Stayers_{j}_1_pl_XX
    if placebo:
        max_t = int(max_T_XX)
        for j in range(1, len(estims) + 1):
            sw_sum = 0.0
            st_sum = 0.0
            for p in range(3, max_t + 1):
                nsw = float(scalars.get(f"N_Switchers_{j}_{p}_pl_XX", 0.0) or 0.0)
                nst = float(scalars.get(f"N_Stayers_{j}_{p}_pl_XX", 0.0) or 0.0)
    
                # Match the R code logic:
                # - add switchers only when there are at least 2 stayers in that pairwise placebo
                # - add stayers only when there is at least 1 switcher in that pairwise placebo
                if nst > 1:
                    sw_sum += nsw
                if nsw > 0:
                    st_sum += nst
    
            scalars[f"N_Switchers_{j}_1_pl_XX"] = sw_sum
            scalars[f"N_Stayers_{j}_1_pl_XX"] = st_sum



    # ----------------------------
    # Placebo table (overall p=1)
    # ----------------------------
    if bool(placebo):
        ret_pl = np.full((len(estims), len(colnames)), np.nan, dtype=float)
        rown_pl = []
        for j, est in enumerate(estims, start=1):
            p = 1
            rown_pl.append(est.upper())
            delta = scalars.get(f"delta_{j}_{p}_pl_XX", np.nan)
            se = scalars.get(f"sd_delta_{j}_{p}_pl_XX", np.nan)
            lb = scalars.get(f"LB_{j}_{p}_pl_XX", np.nan)
            ub = scalars.get(f"UB_{j}_{p}_pl_XX", np.nan)
            ns = scalars.get(f"N_Switchers_{j}_{p}_pl_XX", np.nan)
            nt = scalars.get(f"N_Stayers_{j}_{p}_pl_XX", np.nan)
            ret_pl[j - 1, :] = [delta, se, lb, ub, ns, nt]

        out["table_placebo"] = pd.DataFrame(ret_pl, index=rown_pl, columns=colnames)

    # -----------------------------------------
    # -----------------------------------------
    # Difference test: AOSS vs WAOSS (overall)
    # -----------------------------------------
    if bool(aoss_vs_waoss) and (aoss_XX == 1) and (waoss_XX == 1):
        # R behavior:
        #   diff_delta = delta_aoss - delta_waoss
        #   diff_Phi   = Phi_aoss - Phi_waoss   (one row per ID in IDs_XX)
        #   sd_diff    = sd(diff_Phi)   [or sd(cluster-summed diff_Phi)]
        #   t_stat     = diff_delta * sqrt(n) / sd_diff
        #   pval       = 2 * (1 - pnorm(|t_stat|))
        #   CI         = diff_delta +/- 1.96 * sd_diff / sqrt(n)
        diff = float(scalars.get("delta_1_1_XX", np.nan)) - float(scalars.get("delta_2_1_XX", np.nan))

        diff_phi = pd.to_numeric(IDs_XX.get("Phi_1_XX", np.nan), errors="coerce") - pd.to_numeric(
            IDs_XX.get("Phi_2_XX", np.nan), errors="coerce"
        )

        # If clustering, aggregate influence by cluster first (cluster-sum), like the R code.
        if cluster is not None and "cluster_XX" in IDs_XX.columns:
            tmp = pd.DataFrame({"cluster_XX": IDs_XX["cluster_XX"], "diff_phi": diff_phi})
            diff_phi_used = tmp.groupby("cluster_XX", as_index=False)["diff_phi"].sum()["diff_phi"]
        else:
            diff_phi_used = diff_phi

        diff_phi_used = pd.to_numeric(diff_phi_used, errors="coerce").fillna(0.0)
        n_eff = int(len(diff_phi_used))

        if n_eff <= 1 or not np.isfinite(diff):
            sd_diff = np.nan
            t_stat = np.nan
            pval = np.nan
            lb = np.nan
            ub = np.nan
        else:
            sd_diff = float(diff_phi_used.std(ddof=1))
            # Guard against sd=0
            if (not np.isfinite(sd_diff)) or sd_diff == 0.0:
                t_stat = np.nan
                pval = np.nan
                lb = np.nan
                ub = np.nan
            else:
                t_stat = float(diff * math.sqrt(n_eff) / sd_diff)
                # Normal approximation (R uses pnorm)
                Phi_abs = 0.5 * (1.0 + math.erf(abs(t_stat) / math.sqrt(2.0)))
                pval = float(2.0 * (1.0 - Phi_abs))
                se = float(sd_diff / math.sqrt(n_eff))
                lb = float(diff - 1.96 * se)
                ub = float(diff + 1.96 * se)

        out["aoss_vs_waoss"] = pd.DataFrame(
            {
                "Estimate": [diff],
                "SE": [sd_diff],  # NOTE: R prints SD here (not SE)
                "LB CI": [lb],
                "UB CI": [ub],
                "t stat.": [t_stat],
                "pval.": [pval],
            },
            index=["Diff."],
        )

    return out
